# **Clinical Malaria Reinforced Learnning**

In [0]:
#installing Libraries
!pip install stable-baselines3[extra] gym matplotlib pandas scikit-learn shimmy ipywidgets openpyxl

# Required Libraries

In [0]:

import gym
from gym import spaces
import numpy as np
import pandas as pd
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive, HBox, VBox, Output

#Step 1: Data Preparation

In [0]:
# malaria Dataset
file_path = 'Clinical Main Data.xlsx'
data= pd.read_excel(file_path)


In [0]:
# Preview Data
print(data.head())

In [0]:
# Define features and target variable
features = [
    'chill_cold', 'headache', 'fever', 'generalized body pain',
    'abdominal pain', 'Loss of appetite', 'joint pain', 'vomiting',
    'nausea', 'diarrhea'
]
target = 'Cases'

# Convert TRUE/FALSE to 1/0 for binary classification
data['Cases'] = data['Cases'].astype(int)  # Ensure target variable is numeric
data[features] = data[features].astype(int)  # Convert feature columns to numeric


In [0]:
# Split into train and test sets
X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [0]:
# Save the scaler
scaler_path = "models/scaler.pkl"
os.makedirs("models", exist_ok=True)
joblib.dump(scaler, scaler_path)

# Step 2: Define Custom Environment



In [0]:
class MalariaPredictionEnv(gym.Env):
    def __init__(self, X, y):
        super(MalariaPredictionEnv, self).__init__()
        self.X = X
        self.y = y
        self.current_index = 0

        # Define state and action spaces
        self.observation_space = spaces.Box(low=-1, high=1, shape=(X.shape[1],), dtype=np.float32)
        self.action_space = spaces.Discrete(2)  # Binary classification: 0 (Negative) or 1 (Positive)

    def reset(self):
        self.current_index = 0
        return self.X[self.current_index]

    def step(self, action):
        actual = self.y[self.current_index]
        reward = 1 if action == actual else -1  # Reward for correct prediction
        self.current_index += 1
        done = self.current_index >= len(self.X)
        next_state = self.X[self.current_index] if not done else np.zeros_like(self.X[0])
        return next_state, reward, done, {}

    def seed(self, seed=None):
        np.random.seed(seed)

# Initialize Environment for Training
env = MalariaPredictionEnv(X_train_scaled, y_train.to_numpy())

# Step 3: Train the RL Model

In [0]:
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("models/ppo_malaria")

# Step 4: Load the Model for Continuous Training and Predictions

In [0]:
model = PPO.load("models/ppo_malaria")
scaler = joblib.load(scaler_path)


# Step 5: Create User Input Interface with Widgets

In [0]:
feature_widgets = {
    feature: widgets.ToggleButton(description=feature.replace('_', ' ').title(), value=False)
    for feature in features
}

# Input for actual cases
cases_input = widgets.IntText(value=0, description='Actual Cases:', min=0)

# Output widget for displaying predictions
output = Output()

# Function to handle the input and make a prediction
def handle_submit(change):
    with output:
        output.clear_output()

        # Gather user input
        selected_features = [int(btn.value) for btn in feature_widgets.values()]
        actual_cases = cases_input.value

        # Scale the new input data
        new_data = np.array([selected_features])
        new_scaled_data = scaler.transform(new_data)

        # Predict using the trained model
        prediction = model.predict(new_scaled_data)[0]

        # Convert prediction to a label
        predicted_case = "Positive (1)" if prediction == 1 else "Negative (0)"
        actual_case = "Positive (1)" if actual_cases == 1 else "Negative (0)"

        # Display results
        print("\nSelected Features:")
        for key, value in zip(features, selected_features):
            print(f"{key.replace('_', ' ').title()}: {'Yes' if value else 'No'}")

        print(f"\nPredicted Case: {predicted_case}")
        print(f"Actual Case: {actual_case}")

# Create the submit button and link to the function
submit_button = widgets.Button(description="Submit", layout=widgets.Layout(width="30%", height="40px"))
submit_button.on_click(handle_submit)

# Layout the interface
feature_title = widgets.Label("Select Symptoms (Toggle for Yes/No)")
feature_layout = VBox([
    HBox([feature_widgets[feature] for feature in list(feature_widgets.keys())[i:i+5]])
    for i in range(0, len(feature_widgets), 5)
])

ui = VBox([
    feature_title,
    feature_layout,
    HBox([cases_input]),
    submit_button
])

# Display the UI
display(ui, output)
